In [1]:
import csv
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [2]:
# import fasta file of cds for chimps and gorillas
chimp_cds = SeqIO.to_dict(SeqIO.parse('/Users/toryrichardson/Desktop/Research/chimp_cds.txt','fasta'))
gorilla_cds = SeqIO.to_dict(SeqIO.parse('/Users/toryrichardson/Desktop/Research/gorilla_cds.txt','fasta'))

In [3]:
chimp_data = pd.DataFrame(pd.read_csv('/Users/toryrichardson/Desktop/Research/chimp_genome_data.txt'))
# gets transcript IDs of the longest transcript for each gene
chimp_single_long_transcripts = chimp_data.sort_values("Transcript length (including UTRs and CDS)", ascending=False).drop_duplicates("Gene stable ID")
chimp_long_transcripts = chimp_single_long_transcripts["Transcript stable ID"].tolist()
# gets list of corresponding gene IDs to longest transcripts
chimp_long_genes = chimp_single_long_transcripts["Gene stable ID"].tolist()
# gets data table of genes with the longest transcripts only
chimp_all_long_transcripts = chimp_data[chimp_data["Transcript stable ID"].isin(chimp_long_transcripts)]  

In [4]:
gorilla_data = pd.DataFrame(pd.read_csv('/Users/toryrichardson/Desktop/Research/gorilla_genome_data.txt'))
# gets transcript IDs of the longest transcript for each gene
gorilla_single_long_transcripts = gorilla_data.sort_values("Transcript length (including UTRs and CDS)", ascending=False).drop_duplicates("Gene stable ID")
gorilla_long_transcripts = gorilla_single_long_transcripts["Transcript stable ID"].tolist()
# gets list of corresponding gene IDs to longest transcripts
gorilla_long_genes = gorilla_single_long_transcripts["Gene stable ID"].tolist()
# gets data table of genes with the longest transcripts only
gorilla_all_long_transcripts = gorilla_data[gorilla_data["Transcript stable ID"].isin(gorilla_long_transcripts)] 

In [5]:
chimp_ids = dict(zip(chimp_long_genes, chimp_long_transcripts))
gorilla_ids = dict(zip(gorilla_long_genes, gorilla_long_transcripts))

In [6]:
def get_coordinates(data, sequences, transcriptId):
    exonRows = data[data["Transcript stable ID"]==transcriptId].sort_values("Exon rank in transcript")
    sequence = sequences[transcriptId].seq 
    start_coords = []
    end_coords = []
    if (exonRows["Strand"] < 0).all():
        exonRows = exonRows[::-1]
    for (index, exon) in exonRows.iterrows():
        exon_range = set(range(int(exon["Exon region start (bp)"]), int(exon["Exon region end (bp)"]+1)))
        if not pd.isnull(exon["5' UTR start"]):
            utr5 = set(range(int(exon["5' UTR start"]), int(exon["5' UTR end"]+1)))
            if min(exon_range) < min(utr5):
                start = min(exon_range)
                end = min(utr5)-1
            elif min(exon_range) > min(utr5):
                start = max(utr5)
                end = max(exon_range)
            else:
                start = max(utr5)
                end = max(exon_range)
        elif not pd.isnull(exon["3' UTR start"]):
            utr3 = set(range(int(exon["3' UTR start"]), int(exon["3' UTR end"]+1)))
            if min(exon_range) < min(utr3):
                start = min(exon_range)
                end = min(utr3)-1
            elif min(exon_range) > min(utr3):
                start = max(utr3)
                end = max(exon_range)
            else:
                start = max(utr3)
                end = max(exon_range)
        else:
            start = min(exon_range)
            end = max(exon_range)
        start_coords.append(start)
        end_coords.append(end)
    coords = list(zip(start_coords, end_coords))
    return (coords, sequence)

In [ ]:
%%time
for i in chimp_long_genes:
    temp = get_coordinates(chimp_all_long_transcripts, chimp_cds, str(chimp_ids.get(i)))
    coords = (SeqRecord(temp[-1], id=str(i) + "|" + str(chimp_ids.get(i)), description=str(temp[0])))
    SeqIO.write(coords, "%s.chimp.txt"%chimp_ids.get(i), "fasta")

In [20]:
%%time
for i in gorilla_long_genes:
    temp = get_coordinates(gorilla_all_long_transcripts, gorilla_cds, str(gorilla_ids.get(i)))
    coords = (SeqRecord(temp[-1], id=str(i) + "|" + str(gorilla_ids.get(i)), description=str(temp[0])))
    SeqIO.write(coords, "%s.gorilla.txt"%gorilla_ids.get(i), "fasta")

CPU times: user 8min 21s, sys: 7.13 s, total: 8min 29s
Wall time: 42min 11s
